In [1]:
from generate_3_sat_constraints import solve_with_solver, generate_3_sat_constraints
from real_constraint_utils import solve_smt_formula, cnf_to_smt_over_reals
from pysat.formula import CNF
import pandas as pd
from tqdm import tqdm
from pysat.solvers import Minisat22, Glucose3, Lingeling

# Generate CNF

In [2]:
generate_3_sat_constraints(10, formula_dir="/Users/mihajlobulesnij/Documents/system/RAI/project/DRL_DGM/constraints_generated/sat_constraints")

Generating formulas and benchmarking...


Variables: 100%|██████████| 1/1 [00:12<00:00, 12.97s/it]


✅ Benchmark complete. Results saved to '/Users/mihajlobulesnij/Documents/system/RAI/project/DRL_DGM/constraints_generated/sat_constraints/sat_formulas.csv', formulas saved in '/Users/mihajlobulesnij/Documents/system/RAI/project/DRL_DGM/constraints_generated/sat_constraints/'


# Convert to constraints over Reals

In [6]:
dataset = pd.read_csv("constraints_generated/sat_constraints/sat_formulas.csv")
for filename in tqdm(dataset["filename"]):
    time_mesured_buf = []
    cnf = CNF(from_file=f"constraints_generated/sat_constraints/{filename}")
    cnf_to_smt_over_reals(cnf, filename = "constraints_generated/smt_constraints_random/"+filename[:-4], random_v = True)

100%|██████████| 130/130 [00:00<00:00, 222.16it/s]


Generate data that satisfy constraints with DRL

In [7]:
dataset = pd.read_csv("constraints_generated/sat_constraints/sat_formulas.csv")
for filename in tqdm(dataset["filename"]):
    time_mesured_buf = []
    cnf = CNF(from_file=f"constraints_generated/sat_constraints/{filename}")
    cnf_to_smt_over_reals(cnf, filename = "constraints_generated/smt_constraints/"+filename[:-4], random_v = False)

100%|██████████| 130/130 [00:00<00:00, 228.79it/s]


# Generating Dataset that satisfy constraints

In [4]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm
base_path = "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas_random"
seeds = [1, 2, 3]
python_script = "sample_random_data.py"

info_cnf = pd.read_csv("/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/sat_formulas.csv")
sats = info_cnf.Sat
filenames = info_cnf.filename
commands = []

for file_name, sat in zip(filenames, sats):
    if sat:
        full_constraint_path = os.path.join(base_path, file_name[:-4]+".txt")
        for seed in seeds:
            cmd = f'/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python {python_script} --seed {seed} --constraint_path "{full_constraint_path}"'
            commands.append(cmd)

In [5]:
for i, cmd in tqdm(enumerate(commands, 1)):
    print(f"\n▶️ Running command {i}/{len(commands)}:\n{cmd}\n")
    
    result = subprocess.run(cmd, shell=True)

    if result.returncode != 0:
        print(f"❌ Command failed with exit code {result.returncode}:")
        print(cmd)
        break
    else:
        print(f"✅ Completed {i}/{len(commands)}\n")

0it [00:00, ?it/s]


▶️ Running command 1/276:
/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python DRL_DGM/sample_random_data.py --seed 1 --constraint_path "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas_random/n10_r1.0_i0.txt"

All random seeds set to 1
 *** ALL CONSTRAINTS ***
 *** Normalised Constraints for y_9 ***
10
disjunct 0:  - y_0 > -0.694
disjunct 1:  + y_1 >= 0.609
disjunct 2:  + y_9 >= 0.381

disjunct 0:  + y_1 >= 0.609
disjunct 1:  - y_3 > -0.629
disjunct 2:  - y_9 > -0.281

 + y_9 >= 0.381
 - y_9 > -0.281
disjunct 0:  - y_0 > -0.694
disjunct 1:  + y_1 >= 0.609
disjunct 2:  - y_3 > -0.629

 *** Unnormalised Constraints for y_8 ***
disjunct 0:  - y_2 > -0.262
disjunct 1:  + y_4 >= 0.251
disjunct 2:  + y_7 >= 0.476

disjunct 0:  + y_0 >= 0.794
disjunct 1:  + y_4 >= 0.251
disjunct 2:  + y_8 >= 0.55

disjunct 0:  - y_3 > -0.629
disjunct 1:  - y_6 > -0.505
disjunct 2:  - y_8 > -0.45

disjunct 0:  + y_3 >= 0.729
disjunct 1:  + y_7 >= 0.476
disjunct 2:  + y

1it [00:01,  1.59s/it]

✅ Completed 1/276


▶️ Running command 2/276:
/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python DRL_DGM/sample_random_data.py --seed 2 --constraint_path "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas_random/n10_r1.0_i0.txt"

All random seeds set to 2
 *** ALL CONSTRAINTS ***
 *** Normalised Constraints for y_9 ***
10
disjunct 0:  - y_0 > -0.694
disjunct 1:  + y_1 >= 0.609
disjunct 2:  + y_9 >= 0.381

disjunct 0:  + y_1 >= 0.609
disjunct 1:  - y_3 > -0.629
disjunct 2:  - y_9 > -0.281

 + y_9 >= 0.381
 - y_9 > -0.281
disjunct 0:  - y_0 > -0.694
disjunct 1:  + y_1 >= 0.609
disjunct 2:  - y_3 > -0.629

 *** Unnormalised Constraints for y_8 ***
disjunct 0:  - y_0 > -0.694
disjunct 1:  + y_1 >= 0.609
disjunct 2:  - y_3 > -0.629

disjunct 0:  + y_1 >= 0.609
disjunct 1:  - y_4 > -0.151
disjunct 2:  - y_8 > -0.45

disjunct 0:  - y_0 > -0.694
disjunct 1:  + y_5 >= 0.745
disjunct 2:  - y_8 > -0.45

disjunct 0:  + y_0 >= 0.794
disjunct 1:  + y_4 >= 0.2

2it [00:02,  1.40s/it]

...

✅ Completed 69/276


▶️ Running command 70/276:
/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python DRL_DGM/sample_random_data.py --seed 1 --constraint_path "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas_random/n10_r1.7999999999999998_i3.txt"

All random seeds set to 1
 *** ALL CONSTRAINTS ***
 *** Normalised Constraints for y_9 ***
18
disjunct 0:  - y_3 > -0.566
disjunct 1:  - y_6 > -0.254
disjunct 2:  + y_9 >= 0.426

disjunct 0:  + y_1 >= 0.62
disjunct 1:  - y_7 > -0.259
disjunct 2:  + y_9 >= 0.426

disjunct 0:  - y_2 > -0.151
disjunct 1:  + y_7 >= 0.359
disjunct 2:  - y_9 > -0.326

disjunct 0:  + y_4 >= 0.625
disjunct 1:  + y_6 >= 0.354
disjunct 2:  + y_9 >= 0.426

 + y_9 >= 0.426
 - y_9 > -0.326
 + y_9 >= 0.426
 - y_9 > -0.326
 + y_9 >= 0.426
 - y_9 > -0.326
disjunct 0:  - y_3 > -0.566
disjunct 1:  - y_6 > -0.254
disjunct 2:  - y_2 > -0.151
disjunct 3:  + y_7 >= 0.359

disjunct 0:  + y_4 >= 0.625
disjunct 1:  + y_6 >= 0.354
disjunct 2:  -

70it [01:34,  1.52s/it]

✅ Completed 70/276


▶️ Running command 71/276:
/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python DRL_DGM/sample_random_data.py --seed 2 --constraint_path "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas_random/n10_r1.7999999999999998_i3.txt"

All random seeds set to 2
 *** ALL CONSTRAINTS ***
 *** Normalised Constraints for y_9 ***
18
disjunct 0:  - y_3 > -0.566
disjunct 1:  - y_6 > -0.254
disjunct 2:  + y_9 >= 0.426

disjunct 0:  + y_1 >= 0.62
disjunct 1:  - y_7 > -0.259
disjunct 2:  + y_9 >= 0.426

disjunct 0:  - y_2 > -0.151
disjunct 1:  + y_7 >= 0.359
disjunct 2:  - y_9 > -0.326

disjunct 0:  + y_4 >= 0.625
disjunct 1:  + y_6 >= 0.354
disjunct 2:  + y_9 >= 0.426

 + y_9 >= 0.426
 - y_9 > -0.326
 + y_9 >= 0.426
 - y_9 > -0.326
 + y_9 >= 0.426
 - y_9 > -0.326
disjunct 0:  - y_3 > -0.566
disjunct 1:  - y_6 > -0.254
disjunct 2:  - y_2 > -0.151
disjunct 3:  + y_7 >= 0.359

disjunct 0:  + y_4 >= 0.625
disjunct 1:  + y_6 >= 0.354
disjunct 2:  -

Traceback (most recent call last):
  File "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/DRL_DGM/sample_random_data.py", line 91, in <module>
    check_all_constraints_sat(sampled_data, constraints=constraints, error_raise=True)
  File "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/DRL_DGM/DRL/constraints_code/correct_predictions.py", line 379, in check_all_constraints_sat
    raise Exception('Not satisfied!', constr.readable(), samples_violating_constr[0][3], samples_violating_constr[0][10], samples_violating_constr[0][11])
IndexError: index 10 is out of bounds for dimension 0 with size 10
70it [01:35,  1.37s/it]

❌ Command failed with exit code 1:
/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python DRL_DGM/sample_random_data.py --seed 2 --constraint_path "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas_random/n10_r1.7999999999999998_i3.txt"


/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python DRL_DGM/sample_random_data.py --seed 1 --constraint_path "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas_random/n10_r1.7999999999999998_i0.txt"


### Training

In [2]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm
base_path = "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/smt_formulas_random"
seeds = [1, 2, 3]
python_script = "/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/DRL_DGM/main.py"

info_cnf = pd.read_csv("/Users/mihajlobulesnij/Documents/system/RAI/Constraints_complexity/sat_formulas.csv")
sats = info_cnf.Sat
filenames = info_cnf.filename
commands = []

for file_name, sat in zip(filenames, sats):
    if sat:
        full_constraint_path = os.path.join(base_path, file_name[:-4]+".txt")
        for seed in seeds:
            cmd = f'/opt/homebrew/Caskroom/miniconda/base/envs/drl_env/bin/python {python_script} --seed {42} --constraint_path "{full_constraint_path}" --dataset_index "{seed}"'
            commands.append(cmd)

In [ ]:
for i, cmd in tqdm(enumerate(commands[143:], 1)):
    print(f"\n▶️ Running command {i}/{len(commands)}:\n{cmd}\n")
    
    result = subprocess.run(cmd, shell=True)

    if result.returncode != 0:
        print(f"❌ Command failed with exit code {result.returncode}:")
        print(cmd)
        break
    else:
        print(f"✅ Completed {i}/{len(commands)}\n")
